### Improved RAG
- [ ] PDF URL loader
- [ ] new chunking method
- [ ] 
- [ ] Semantic chunking 
- [ ] Rerank retrieval result by LLM
- [ ] Rewrite query
- [ ] Input with HTML, url

In [1]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
import numpy as np
from sklearn.manifold import TSNE
import glob
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import chromadb
from langchain_chroma import Chroma


load_dotenv(override=True)

True

In [2]:
import gradio as gr
print(gr.__version__)
# Gradio 6 onwards will generate hisotry in the format of [{"role": "user", "content": "..."}, ...]
# Gradio 5 and below will generate history in the format of [{"text": "..."}, ...]


5.49.1


In [3]:
DB_NAME = "chroma_db"
DATA_DIR = '../data' 
MODEL = "gpt-4o-mini"
TOP_K = 5

In [4]:
# load docs
# DATA_DIR = '../data'  # Remove the * - it's not part of the path

# docs=[]
# # Convert generator to list so we can use len() and iterate multiple times
# files = list(Path(DATA_DIR).glob('*'))
# print(f"Found {len(files)} files in the knowledge base")
# print(f"Files: {[file.name for file in files]}")

# for file in files:
#     if file.is_file():
#         print("Loading file: ", file)
#         with open(file, "r", encoding="utf-8") as f:
#             docs.append({"text": f.read(), "source": file.name, "type": file.suffix})

# print(f"\nLoaded {len(docs)} documents")
# print(docs)


documents = []

doc_type = os.path.basename(DATA_DIR) # get the name of the folder. employees, contracts, products, company
loader = DirectoryLoader(DATA_DIR, glob="*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
folder_docs = loader.load()
for doc in folder_docs:
    doc.metadata["doc_type"] = doc_type
    doc.metadata["source"] = doc.metadata.get("source").split("/")[-1]
    documents.append(doc) # Has to be Document object !!!

print(documents)


[Document(metadata={'source': 'cv.md', 'doc_type': 'data'}, page_content="LI BEIJI\n +65 8432 9134     http://www.linkedin.com/in/beiji-li     libeiji08121999@gmail.com   https://github.com/SuperiormonLBJ    https://portfolio-beiji.vercel.app/\nSUMMARY\nPassionate Software Engineer and AI Innovator with a track record of bridging cutting-edge AI solutions with a solid software engineering base. Skilled in designing and leading full-stack platforms and intelligent automation systems, drive real-world impact and unlock actionable AI insights.\n \nWORK EXPERIENCE\nSoftware Engineer\tAug 2023 - Present\nUnited Overseas Bank \n•\tLed adoption and production rollout of n8n, empowering 10+ teams to build customized automation workflow or agentic solution with 10k SGD saved per month.\n•\tDesigned and implemented Prompt Template Hub(React & Spring Boot) with Bitbucket integration, providing an interactive dashboard to all domains with version control and access control for 200+ templates.\n•\t

### Improvement 1: PDF/URL Loader

In [42]:
from langchain_community.document_loaders import PlaywrightURLLoader, PyPDFLoader, UnstructuredHTMLLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

def load_document_pdf() -> str:
    documents = []
    doc_type = os.path.basename(DATA_DIR)
    loader = DirectoryLoader(DATA_DIR, glob="*.pdf", loader_cls=PyPDFLoader)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        doc.metadata["source"] = doc.metadata.get("source").split("/")[-1]
        documents.append(doc)
    
    return documents

from langchain_community.document_loaders import WebBaseLoader
from urllib.parse import urlparse
from pathlib import Path

LINKEDIN_PROMPT = """
You are a text cleaning and summarization assistant. 

I will provide you with a raw text dump from a LinkedIn page. Your task is to:

1. Remove all unnecessary website text, boilerplate, and repeated login/signup prompts.
2. Remove all \n, excessive spaces, and strange characters.
3. Extract only the **personal profile information** of the user, including:
   - Name
   - Headline / Summary
   - Location
   - Connections / Followers (optional)
   - Current Experience (Job Title + Company + Years)
   - Past Experience (Job Title + Company + Years)
   - Education (School + Degree + Years)
   - Certifications or Courses
   - Languages
   - Projects or notable achievements
4. Present the information in a **clean, readable, structured format**, like:
   
   Name: ...
   Headline: ...
   Location: ...
   Current Experience: ...
   Past Experience: ...
   Education: ...
   Certifications: ...
   Languages: ...
   Projects: ...
   
5. Ignore any text that is not part of the user profile (e.g., "Sign in", "Join now", LinkedIn navigation, footer, ads, etc.).
"""

"""TODO: better way to handle html content, can not use "PlaywrightURLLoader since not supported in py3.12"""
def load_document_url():
    """
    Load documents from a file containing URLs and save them as text files
    """
    documents = []

    urls_file = Path(DATA_DIR) / "urls.txt"
    if not urls_file.exists():
        return documents

    with open(urls_file) as f:
        urls = [u.strip() for u in f.readlines() if u.strip()]

    if not urls:
        return documents

    loader = WebBaseLoader(urls)
    loaded_docs = loader.load()

    for doc in loaded_docs:
        doc.page_content = doc.page_content.encode("latin1", errors="ignore").decode("utf-8", errors="ignore")
        parsed = urlparse(doc.metadata.get("source", ""))
        doc.metadata["doc_type"] = "url"
        doc.metadata["source"] = parsed.netloc
        if doc.metadata.get("source") == "www.linkedin.com":
            llm = ChatOpenAI(model=MODEL, temperature=0)
            messages = [SystemMessage(content=LINKEDIN_PROMPT.format(raw_linkedin_text=doc.page_content))]
            messages.append(HumanMessage(content=doc.page_content))
            doc.page_content = llm.invoke(messages).content
        doc.metadata["full_url"] = doc.metadata.get("source")
        documents.append(doc)
        with open(Path(DATA_DIR) / (doc.metadata.get("source") + ".txt"), "w") as f:
            f.write(doc.page_content)

    print(f"Loaded {len(documents)} URL documents")
    return documents

In [7]:
load_document_pdf()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)


[Document(metadata={'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20260122073854Z00'00'", 'title': 'Resume_LI_BEIJI', 'author': 'beiji M4', 'moddate': "D:20260122073854Z00'00'", 'source': 'Resume_LI_BEIJI.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'doc_type': 'data'}, page_content="LI BEIJI +65 8432 9134    Linkedin    libeiji08121999@gmail.com  Github   Portfolio SUMMARY Passionate Software Engineer and AI Innovator with a track record of bridging cutting-edge AI solutions with a solid software engineering base. Skilled in designing and leading full-stack platforms and intelligent automation systems, drive real-world impact and unlock actionable AI insights.   WORK EXPERIENCE Software Engineer Aug 2023 - Present United Overseas Bank  • Led adoption and production rollout of n8n, empowering 10+ teams to build customized automation workflow or agentic solution with 10k SGD saved per month. • Designed and implemented Pr

In [43]:
load_document_url()

Loaded 2 URL documents


[Document(metadata={'source': 'portfolio-beiji.vercel.app', 'title': "Beiji's Blog", 'description': 'Personal thoughts and insights on technology, design, and life.', 'language': 'en', 'doc_type': 'url', 'full_url': 'portfolio-beiji.vercel.app'}, page_content="Beiji's Blogʵʵ.mp3 - Deca JoinsLi BeijiHomeContactYour browser does not support the video tag...WelcomeLi Beiji/˥Hi! This incredible journey begins with obsessing over the PC games and getting addicted to the whole computer world in the end. Now I'm a passionate Software Engineer and AI innovator, focused on creating meaningful digital experiences.BACK TO HOMEFeatured ProjectsSep 15, 2023Terminal-Based TransactionManagementSystem for BankA terminal-based banking system built with Java and PostgreSQL, featuring role-based access, secure authentication, transaction management, and a user-friendly CLI interface with audio feedback.Read MoreBackendOct 15, 2023Role-Based Banking Management SystemA secure banking management system with

In [17]:
documents.extend(load_document_url())
documents.extend(load_document_pdf())


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)


Loaded 2 URL documents


In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"Divided into {len(chunks)} chunks")
print(f"First chunk:\n\n{chunks[0]}")
print(f"Second chunk:\n\n{chunks[1]}")
print(f"Last chunk:\n\n{chunks[-1]}")

Divided into 118 chunks
First chunk:

page_content='LI BEIJI
 +65 8432 9134     http://www.linkedin.com/in/beiji-li     libeiji08121999@gmail.com   https://github.com/SuperiormonLBJ    https://portfolio-beiji.vercel.app/
SUMMARY
Passionate Software Engineer and AI Innovator with a track record of bridging cutting-edge AI solutions with a solid software engineering base. Skilled in designing and leading full-stack platforms and intelligent automation systems, drive real-world impact and unlock actionable AI insights.
 
WORK EXPERIENCE' metadata={'source': 'cv.md', 'doc_type': 'data'}
Second chunk:

page_content='WORK EXPERIENCE
Software Engineer	Aug 2023 - Present
United Overseas Bank 
•	Led adoption and production rollout of n8n, empowering 10+ teams to build customized automation workflow or agentic solution with 10k SGD saved per month.
•	Designed and implemented Prompt Template Hub(React & Spring Boot) with Bitbucket integration, providing an interactive dashboard to all domains wit

In [21]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings,collection_name="my_collection").delete_collection()

vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=DB_NAME,
    collection_name="my_collection"
)

print(f"Vectorstore created with {vector_store._collection.count()} documents")

collection = vector_store._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

Vectorstore created with 118 documents
There are 118 vectors with 3,072 dimensions in the vector store


In [22]:
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings, collection_name="my_collection")
retriever = vectorstore.as_retriever()
retriever.invoke("What is Beiji's current job based on CV?")

[Document(id='e294471f-5f25-4225-9bb8-29b3a85e46d3', metadata={'source': 'www.linkedin.com', 'description': 'Passionate Software Engineer and AI Innovator with a proven ability to bridge‚Ä¶ ¬∑ Experience: UOB ¬∑ Education: National University of Singapore ¬∑ Location: Singapore ¬∑ 283 connections on LinkedIn. View Beiji Li‚Äôs profile on LinkedIn, a professional community of 1 billion members.', 'language': 'en', 'full_url': 'www.linkedin.com', 'doc_type': 'url', 'title': 'Beiji Li - UOB | LinkedIn'}, page_content='******** ********\n      \n\n\n\n\n\n\n\n\n\n\n\n          \n      ******** ********** ** *********\n    \n        \n\n\n        ****** ** *********** * **** undefined 4.62/5.0\n      \n\n\n\n\n2021 - 2023\n \n\n\n\n\n\n\n\n\n\n          \n      ********* **********\n    \n        \n\n\n        ******** ** *********** * ** undefined ********\n      \n\n\n\n\n2017 - 2021\n \n\n\n\n\n\n\n\nView Beiji‚Äôs full experience\nSee their title, tenure and more.\n\n\n\n\n \n          

In [23]:
# Check all documents in the collection
collection = vectorstore._collection
all_docs = collection.get(include=["documents", "metadatas"])

# Count by source
from collections import Counter
sources = [meta.get("source", "unknown") for meta in all_docs["metadatas"]]
print("Documents by source:", Counter(sources))

# Check for duplicates
doc_contents = all_docs["documents"]
print(f"Total documents: {len(doc_contents)}")
print(f"Unique documents: {len(set(doc_contents))}")

Documents by source: Counter({'www.linkedin.com': 91, 'cv.md': 8, 'Resume_LI_BEIJI.pdf': 8, 'hobby.md': 6, 'portfolio-beiji.vercel.app': 5})
Total documents: 118
Unique documents: 117


### RAG - Retrieval

In [24]:
from langchain_openai import ChatOpenAI


vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings, collection_name="my_collection")
retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})

llm=ChatOpenAI(model=MODEL) # Langchain wrapper for OpenAI

System_prompt = """
You are a helpful assistant that can answer questions about the user's CV and hobbies.
You are given a question and a context.
You need to answer the question based on the context.
Context:
{context}
"""

In [26]:
from langchain_core.messages import HumanMessage, SystemMessage, convert_to_messages


def fetch_context(query):
    """
    Fetch the context for the query from the vector store with Top K results
    """
    context = retriever.invoke(query)
    return context

def combine_all_user_questions(question: str, history: list) -> str:
    """
    Combine all user questions into a single question 
    to solve the issue that user ask "what did she do before"
    """
    prior = "\n".join(msg["content"] for msg in history if msg.get("role") == "user")
    
    # Handle empty prior case
    if prior:
        return prior + "\n" + question
    else:
        return question

def generate_answer(query, history: list):
    """
    Generate the answer for the query

    history from gradio is a list of dictionaries with keys "role" and "content" this is OpenAI format, need to convert it to the format that langchain can use
    """
    
    print(f"DEBUG - query type: {type(query)}, value: {query}")
    print(f"DEBUG - history type: {type(history)}, length: {len(history) if isinstance(history, list) else 'not a list'}")
    # ... rest of function

    # query = query["text"]
    
    # combine all user questions into a single question
    combined_question = combine_all_user_questions(query, history)
    context_docs = fetch_context(combined_question)
    context = "\n".join([doc.page_content for doc in context_docs])

    # convert history to langchain format
    messages = [SystemMessage(content=System_prompt.format(context=context))]
    print("history:", history)
    print("convert_to_messages(history):", convert_to_messages(history))
    print("messages before extend:", messages)
    messages.extend(convert_to_messages(history)) # add history to the messages, convert_to_messages: convert the history to messages, from LangChain's format to OpenAI's format, import from langchain_core.messages
    print("messages after extend:", messages)
    messages.append(HumanMessage(content=query))
    print("messages after append human message:", messages)

    response = llm.invoke(messages)

    return response.content, context_docs

### Frontend with Gradio first for testing

In [ ]:
import gradio as gr
print(gr.__version__)

def format_context(context):
    result = "<h2 style='color: #ff7800;'>Relevant Context</h2>\n\n"
    for doc in context:
        result += f"<span style='color: #ff7800;'>Source: {doc.metadata['source']}</span>\n\n"
        result += doc.page_content + "\n\n"
    return result


def chat(history):
    """History is in messages format: [{"role": "user", "content": "..."}, ...]"""
    if not history:
        return history, ""
    
    last_message = history[-1]["content"]
    prior = history[:-1]
    answer, context = generate_answer(last_message, prior)
    
    # Create new history instead of modifying in place
    new_history = history + [{"role": "assistant", "content": answer}]
    return new_history, format_context(context)


def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

theme = gr.themes.Soft(font=["Inter", "system-ui", "sans-serif"])

with gr.Blocks(title="Best Friend of Beiji", theme=theme) as ui:
    gr.Markdown("# 🏢 Best Friend of Beiji\nAsk me anything about Beiji!")

    with gr.Row():
        with gr.Column(scale=1):
            chatbot = gr.Chatbot(
                    label="💬 Conversation", height=600, type="messages", show_copy_button=True
                )
            message = gr.Textbox(
                label="Your Question",
                placeholder="Ask anything about Beiji...",
                show_label=False,
            )

        with gr.Column(scale=1):
            context_markdown = gr.Markdown(
                label="📚 Retrieved Context",
                value="*Retrieved context will appear here*",
                container=True,
                height=600,
            )

    message.submit(
        put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]
    ).then(chat, inputs=chatbot, outputs=[chatbot, context_markdown])

ui.launch(inbrowser=True)

5.49.1
* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


python(40958) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


DEBUG - query type: <class 'str'>, value: what is beiji's job
DEBUG - history type: <class 'list'>, length: 0
history: []
convert_to_messages(history): []
messages before extend: [SystemMessage(content="\nYou are a helpful assistant that can answer questions about the user's CV and hobbies.\nYou are given a question and a context.\nYou need to answer the question based on the context.\nContext:\nBeiji is, at his core, an innovator and a proactive problem-solver. He is always experimenting, rigorously testing new ideas, and consistently looking for smarter, more efficient solutions rather than merely accepting the status quo. His impressive ability to rapidly pick up advanced computer-vision techniques, seamlessly apply them to challenging civil-engineering contexts, and produce credible results speaks volumes about his drive, intellectual curiosity, and remarkable learning speed.\nWhat truly sets Beiji apart is his innate drive to constantly push boundaries. He doesn't just solve probl